In [1]:
sc

In [1]:
import time, sys, cherrypy, os
from paste.translogger import TransLogger
from app import create_app
from pyspark import SparkContext, SparkConf
 
def init_spark_context():
    # load spark context
    conf = SparkConf().setAppName("movie_recommendation-server")
    # IMPORTANT: pass aditional Python modules to each worker
    sc = SparkContext(conf=conf, pyFiles=['engine.py', 'app.py'])
 
    return sc
 
 
def run_server(app):
 
    # Enable WSGI access logging via Paste
    app_logged = TransLogger(app)
 
    # Mount the WSGI callable object (app) on the root directory
    cherrypy.tree.graft(app_logged, '/')
 
    # Set the configuration of the web server
    cherrypy.config.update({
        'engine.autoreload.on': True,
        'log.screen': True,
        'server.socket_port': 5432,
        'server.socket_host': '0.0.0.0'
    })
 
    # Start the CherryPy WSGI web server
    cherrypy.engine.start()
    cherrypy.engine.block()
 
 
if __name__ == "__main__":
    # Init spark context and load libraries
    #sc = init_spark_context()
    dataset_path = os.path.join('datasets', 'ml-latest')
    app = create_app(sc, dataset_path)
 
    # start web server
    run_server(app)

INFO:engine:Starting up the Recommendation Engine: 
INFO:engine:Loading Ratings data...
INFO:engine:Loading Movies data...
INFO:engine:Counting movie ratings...
INFO:engine:Training the ALS model...


Py4JJavaError: An error occurred while calling o102.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 43, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.recommendation.ALS$UncompressedInBlockSort.allocate(ALS.scala:1153)
	at org.apache.spark.ml.recommendation.ALS$UncompressedInBlockSort.allocate(ALS.scala:1102)
	at org.apache.spark.util.collection.TimSort$SortState.ensureCapacity(TimSort.java:951)
	at org.apache.spark.util.collection.TimSort$SortState.mergeLo(TimSort.java:699)
	at org.apache.spark.util.collection.TimSort$SortState.mergeAt(TimSort.java:525)
	at org.apache.spark.util.collection.TimSort$SortState.mergeCollapse(TimSort.java:453)
	at org.apache.spark.util.collection.TimSort$SortState.access$200(TimSort.java:325)
	at org.apache.spark.util.collection.TimSort.sort(TimSort.java:153)
	at org.apache.spark.util